In [11]:
import random

class Generation_data:
    def __init__(self, nb_objets, nbres_camions):
        self.nb_objets = nb_objets
        self.nbres_camions = nbres_camions
        self.grand_tableau = self.generer_tableau()
        self.tableau_camion = self.gen_tab_camion()

    def generer_tableau(self):
        nb_villes = self.nb_objets * 2
        tab_ville = list(range(1, nb_villes + 1))
        grand_tableau = []

        for _ in range(nb_villes // 2):
            ville_collecte = tab_ville.pop(random.randint(0, len(tab_ville) - 1))
            ville_livraison = tab_ville.pop(random.randint(0, len(tab_ville) - 1))
            camion = random.randint(1, self.nbres_camions)
            grand_tableau.append([ville_collecte, ville_livraison, [camion]])

        return grand_tableau

    def gen_tab_camion(self):
        tab_camion = [[] for _ in range(self.nbres_camions)]

        for route in self.grand_tableau:
            camion_index = route[2][0] - 1
            tab_camion[camion_index].extend([route[0], route[1]])

        return tab_camion

    def affichage_camions(self):
        for i, camions in enumerate(self.tableau_camion):
            print(f"Camion {i + 1} : {camions}")

    def affichage_1_camion(self, id):
        if 1 <= id <= self.nbres_camions:
            print(f"Truck {id} : {self.tableau_camion[id - 1]}")
        else:
            print("Invalid truck ID")

In [12]:
nombres_objets=10
nombres_camions=3
logistics = Generation_data(nombres_objets, nombres_camions)
print()
print(logistics.tableau_camion)

logistics.affichage_camions()



[[1, 19, 17, 10, 4, 16, 11, 8, 14, 13], [18, 7, 20, 15, 9, 6], [12, 3, 2, 5]]
Camion 1 : [1, 19, 17, 10, 4, 16, 11, 8, 14, 13]
Camion 2 : [18, 7, 20, 15, 9, 6]
Camion 3 : [12, 3, 2, 5]


In [6]:
from geopy.geocoders import Nominatim
import requests
import numpy as np
from tqdm import tqdm
import pandas as pd

class DataGenCarto:
    def __init__(self, cities):
        self.cities = cities
        self.coordinates = self.get_coordinates()

    def get_coordinates(self):
        geolocator = Nominatim(user_agent="city_distance_calculator")
        coordinates = []
        for city in tqdm(self.cities, desc="Obtention des coordonnées"):
            location = geolocator.geocode(f"{city}, France")
            if location:
                coordinates.append((location.longitude, location.latitude))
            else:
                coordinates.append(None)
        return coordinates

    def get_distance_osrm(self, start_coords, end_coords):
        start = f"{start_coords[0]},{start_coords[1]}"
        end = f"{end_coords[0]},{end_coords[1]}"
        url = f'http://router.project-osrm.org/route/v1/driving/{start};{end}?overview=false'
        response = requests.get(url)
        data = response.json()
        if data['code'] == 'Ok':
            distance = data['routes'][0]['distance'] / 1000  # distance en kilomètres
            return distance
        else:
            return None

    def calculate_distances_matrix(self):
        num_cities = len(self.cities)
        distances = np.zeros((num_cities, num_cities))

        for i in tqdm(range(num_cities), desc="Calcul des distances"):
            for j in range(i + 1, num_cities):
                if self.coordinates[i] and self.coordinates[j]:
                    distance = self.get_distance_osrm(self.coordinates[i], self.coordinates[j])
                    distances[i][j] = distance
                    distances[j][i] = distance

        return distances

    def save_to_excel(self, distances_matrix, file_name="distances_villes.xlsx"):
        df = pd.DataFrame(distances_matrix, index=self.cities, columns=self.cities)
        df.to_excel(file_name)
        print(f"Le tableau des distances a été enregistré dans le fichier Excel '{file_name}'.")



ModuleNotFoundError: No module named 'geopy'

In [5]:
# Exemple d'utilisation
cities = ["Paris", "Lyon", "Parempuyre", "Marseille", "Bordeaux", "Rennes", "Reims", "Grenoble", "Toulon", "Angers", 
          "Goudou", "Carcassonne", "Condom", "Lille", "Nice", "Toulouse", "Nantes", "La Rochelle", "Strasbourg", 
          "Montpellier", "Tourcoing", "Brives", "Pauillac", "St-Vincent de Pertignas", "Maubeuge", "Plouhinec", 
          "Prayssas", "Montcuq", "Anus", "St-Etienne", "Poil", "Bénisson-Dieu", "Avignon", "Vichy", "VilleJuif", 
          "La Tombe", "Monteton", "Montastruc-la-Conseillère", "Le Havre", "Roubaix", "Langon", "Saint-Maur-des-Fossés", 
          "Glaire", "Royan", "Dijon", "Caen", "Saint-Étienne", "Orléans", "Clermont-Ferrand", "Limoges", "Perpignan", 
          "Besançon", "Metz", "Amiens", "Annecy", "Belfort", "Bourg-en-Bresse", "Chartres", "Colmar", "Dax", "Épinal", 
          "Fréjus", "Gap", "Hyères", "Istres", "Laval", "Lorient", "Mâcon", "Narbonne", "Orange", "Périgueux", "Quimper", 
          "Roanne", "Saint-Lô", "Tarbes", "Troyes", "Valence", "Vierzon", "Cahors", "Brive-la-Gaillarde", "Sète", 
          "Pamiers", "Foix", "Millau", "Alès", "Montauban", "Aurillac", "Auch", "Albi", "Cahors", "Agen", "Laon", 
          "Saint-Quentin", "Beauvais", "Châteauroux", "Blois", "Saint-Nazaire", "Niort", "Béziers", "Castres", 
          "Chambéry", "Thonon-les-Bains", "Bayonne", "Saint-Jean-de-Luz", "Tarbes", "Mont-de-Marsan", "Dole", 
          "Chalon-sur-Saône", "Lons-le-Saunier", "Vienne", "Mende", "Carpentras", "Gap", "Briançon", "Grasse", 
          "Manosque", "Forcalquier", "L'Isle-sur-la-Sorgue", "Bourgoin-Jallieu", "Villefranche-sur-Saône"]

data_gen_carto = DataGenCarto(cities)
distances_matrix = data_gen_carto.calculate_distances_matrix()
print("Tableau des distances (en km) :")
print(distances_matrix)

data_gen_carto.save_to_excel(distances_matrix)


NameError: name 'DataGenCarto' is not defined

In [8]:
import pandas as pd

class DataExtract:
    def __init__(self, file_path):
        self.file_path = file_path
        self.villes = []
        self.distances = []

    def extract_distance_data(self, z):
        # Lire le fichier Excel
        df = pd.read_excel(self.file_path, index_col=0)
        
        # Extraire les noms des villes et les tronquer à la taille z
        self.villes = df.columns.tolist()[:z]
        
        # Extraire les distances dans un tableau 2D et tronquer à la taille z x z
        self.distances = df.values.tolist()[:z]
        self.distances = [row[:z] for row in self.distances]

    def multiply_distances(self, k):
        # Multiplier chaque élément du tableau par le coefficient k et convertir en entier
        self.distances = [[int(distance * k) for distance in row] for row in self.distances]

    def display_data(self):
        print("Villes:", self.villes)
        print("Distances:", self.distances)




ModuleNotFoundError: No module named 'openpyxl'

In [13]:
# Exemple d'utilisation
file_path = 'distances_villes_medoc.xlsx'
z = 3  # Taille du tableau souhaitée
if z > 50:
    print("Z supérieur a 50 doc z = 50")
    z = 50
k = 0.8  # Coefficient de multiplication
print("data")
data_extract = DataExtract(file_path)
data_extract.extract_distance_data(z)
data_extract.multiply_distances(k)
data_extract.display_data()

data
Villes: ['Bordeaux', 'Pauillac', 'Saint-Estèphe']
Distances: [[0, 42, 138], [42, 0, 177], [138, 177, 0]]
